In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from DifferentialEvolution import DifferentialEvolution
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from copy import deepcopy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
LIMITS = [(-10,10)]*2

PARAMS = {
    "popsize":30,
    "iterations":100,
    "mut":0.5,
    "crossp":0.7,
    "generation_history":True,
    "track_best":True,
}

def squares(x):
    return sum(i*i for i in x)

currentToBest = DifferentialEvolution(squares, LIMITS)
randMutation = DifferentialEvolution(squares, LIMITS)

In [23]:
INITIAL_POPULATION = 1/10 * np.mgrid[1:10,1:10]
INITIAL_POPULATION = np.rec.fromarrays(INITIAL_POPULATION).flatten()
INITIAL_POPULATION = np.array([np.array(list(pt)) for pt in INITIAL_POPULATION])

In [24]:
currentToBest.optimize(**PARAMS, initial_pop=INITIAL_POPULATION.copy())
randMutation.optimize(**PARAMS, initial_pop=INITIAL_POPULATION.copy())

(array([0., 0.]), 0.0)

In [55]:
def make_mesh(limits):
    x0, x1 = limits[0]
    y0, y1 = limits[1]
    
    x = np.arange(x0, x1 + 0.1, 0.5)
    y = np.arange(y0, y1 + 0.1, 0.5)
    
    xs, ys = np.meshgrid(x,y)
    return xs, ys, x, y

In [59]:
xs, ys, x, y = make_mesh(LIMITS)

z = xs**2 + ys**2

fig = make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': False}]],)
fig.add_trace(
    go.Surface(
        x=x,y=y,z=z, opacity=0.3,showscale=False
        ),
    1,1
)
fig.add_trace(
    go.Contour(
        z=z, opacity=0.3, showscale=False, x0=-10, dx=0.5, y0=-10, dy=0.5
        ),
    1,2
)

for i, gen in enumerate(currentToBest.generation_history):
    if i < 20 and (i < 2 or i % 5 == 0):
        ptsx = np.array([x for x, _ in gen])
        ptsy = np.array([y for _, y in gen])
        ptsz = ptsx**2 + ptsy**2
        fig.add_trace(
            go.Scatter3d(
                x=ptsx,
                y=ptsy, 
                z=ptsz, 
                mode='markers',
                marker=dict(size=4),
                name=f'Generation {i}'
            ), 1, 1
        )
        fig.add_trace(
            go.Scatter(
                x=ptsx,
                y=ptsy, 
                #z=ptsz, 
                mode='markers',
                marker=dict(size=6),
                name=f'Generation {i}'
            ), 1, 2
        )

        

fig.update_layout(width=1400, height=700, showlegend=True)
fig.show()

In [37]:
xs, ys, x, y = make_mesh(LIMITS)

z = xs**2 + ys**2

fig = go.Figure()
fig.add_trace(go.Surface(x=x,y=y,z=z, opacity=0.3,showscale=False))

for i, gen in enumerate(randMutation.generation_history):
    if i < 20 and (i < 5 or i % 3 == 0):
        ptsx = np.array([x for x, _ in gen])
        ptsy = np.array([y for _, y in gen])
        ptsz = ptsx**2 + ptsy**2
        fig.add_trace(
            go.Scatter3d(
                x=ptsx,
                y=ptsy, 
                z=ptsz, 
                mode='markers',
                marker=dict(size=4),
                name=f'Generation {i}'
            )
        )
        

fig.update_layout(width=700, height=700, showlegend=True)
fig.show()

In [116]:
points

array([[0.2, 0.2],
       [0.2, 0.4],
       [0.2, 0.6],
       [0.2, 0.8],
       [0.4, 0.2],
       [0.4, 0.4],
       [0.4, 0.6],
       [0.4, 0.8],
       [0.6, 0.2],
       [0.6, 0.4],
       [0.6, 0.6],
       [0.6, 0.8],
       [0.8, 0.2],
       [0.8, 0.4],
       [0.8, 0.6],
       [0.8, 0.8]])

In [117]:
test = DifferentialEvolution(squares, LIMITS)
test.optimize(**PARAMS, initial_pop=points)

[[0.2 0.2]
 [0.2 0.4]
 [0.2 0.6]
 [0.2 0.8]
 [0.4 0.2]
 [0.4 0.4]
 [0.4 0.6]
 [0.4 0.8]
 [0.6 0.2]
 [0.6 0.4]
 [0.6 0.6]
 [0.6 0.8]
 [0.8 0.2]
 [0.8 0.4]
 [0.8 0.6]
 [0.8 0.8]]


(array([0.0000000e+00, 8.8817842e-16]), 7.888609052210118e-31)

In [118]:
xs, ys, x, y = make_mesh(LIMITS)

z = xs**2 + ys**2

fig = go.Figure()
fig.add_trace(go.Surface(x=x,y=y,z=z, opacity=0.6,showscale=False))

for i, gen in enumerate(test.generation_history):
    if i < 10:
        ptsx = np.array([x for x, _ in gen])
        ptsy = np.array([y for _, y in gen])
        ptsz = ptsx**2 + ptsy**2
        fig.add_trace(go.Scatter3d(x=ptsx, y=ptsy, z=ptsz, mode='markers',name=f'Generation {i}'))
        

fig.update_layout(width=700, height=700, showlegend=True)
fig.show()

In [98]:
np.random.rand(20, 2)

array([[0.20419576, 0.15551923],
       [0.07880364, 0.52850203],
       [0.4207786 , 0.46012917],
       [0.83971668, 0.71210987],
       [0.82221703, 0.5130811 ],
       [0.09788136, 0.13127718],
       [0.24121057, 0.62950136],
       [0.87699102, 0.57528389],
       [0.13076141, 0.50545699],
       [0.35153331, 0.33185602],
       [0.56164558, 0.38327806],
       [0.1186969 , 0.36579835],
       [0.34591019, 0.97303617],
       [0.4801703 , 0.95560035],
       [0.11184244, 0.16091025],
       [0.13047821, 0.8448305 ],
       [0.58247745, 0.77625867],
       [0.08002415, 0.33094083],
       [0.27314271, 0.08339399],
       [0.91458371, 0.19827557]])